# mmperfmon demo

This notebook graphs the topology of a Spectrum Scale cluster based on the output of `mmperfmon query` using the mmperfmon connector.  Requires the `python-graphviz` package from Anaconda.

In [ ]:
%matplotlib inline

In [ ]:
import os
import glob

In [ ]:
import graphviz

In [ ]:
import tokio

## Specify location of mmperfmon input

The following input file can be a single output of mmperfmon, a directory full of mmperfmon output files, or a tarball containing mmperfmon outputs.  The `gpfs_nsdds_bytes_written` _must_ be present in the file though.

In [ ]:
MMPERFMON_INPUT_FILE = os.path.join("..", "tests", "inputs", "mmperfmon.2019-05-15-micro.tgz")

In [ ]:
mmpm = None

In [ ]:
#tokio.connectors.mmperfmon.Mmperfmon.from_file()
for file in glob.glob(MMPERFMON_INPUT_FILE):
    if mmpm is None:
        mmpm = tokio.connectors.mmperfmon.Mmperfmon.from_file(file)
    else:
        mmpm.load(file)

In [ ]:
df = mmpm.to_dataframe_by_metric('gpfs_nsdds_bytes_written')

In [ ]:
server_types = set([])
for col in df.columns:
    server_types.add(col)

## Extract basic insight from mmperfmon output

In [ ]:
servers = set([])
luns = set([])
for col in df.columns:
    server, lun = col.split(':')
    servers.add(server)
    luns.add(lun)

print("Found:")
print("  %3d servers" % len(servers))
print("  %3d luns" % len(luns))

In [ ]:
print("Found %d unique server-lun combinations" % len(df.columns))

## Build the cluster topology graph

In [ ]:
edges = {}

# neato, fdp, circo
dot = graphviz.Graph(engine='fdp', format='png')
dot.attr(outputorder="edgesfirst")

# comment out below to show node labels
dot.attr('node', label="")

for server_type in sorted(list(server_types)):
    server, lun = server_type.split(':')
    dot.edge(server, lun, color="#00000044")
    dot.node(lun, shape='oval', style="filled", fillcolor="firebrick")
    dot.node(server, shape='box', style="filled", fillcolor="grey")


In [ ]:
dot.render('gpfs-topology.gv', view=True)